In [1]:
import pandas as pd
import re
from os import listdir
from tqdm import tqdm

In [2]:
def fix(group):
  if len(group) < 1:
    return group

  if group.values[-1] in ['ขาท', 'ขบาท']:
    group.values[-1] = 'บาท'
  if group.values[-1] == 'บาท':
    group.values[-2] = group.values[-2].replace(',', '').replace('.', '')
  
  if re.match('^0\d{4}', group.values[0]):
    temp = ''
    for t in group.values[0]:
      if t == '-': return group
      if t != ' ': temp += t
      if len(temp) == 5: temp += '-'
    group.values[0] = temp

  return group


In [28]:
dfs = []
rootDir = './bud-csv/'
for file in tqdm(sorted(listdir(rootDir))):
    if not file.endswith('_raw.tsv'): continue

    df = pd.read_csv(rootDir+file, sep='\t')\
        .sort_values(by=['filename', 'line_num'])\
        .reset_index(drop=True)\
        .dropna()

    df['text'] = df.groupby(['filename', 'line_num']).text.apply(fix)

    if 'page' not in df.columns:
        df['page'] = df.filename.apply(lambda x: int(x[:x.rfind('.')]))
    
    df['filename'] = df.filename.apply(lambda x: file[:file.rfind('_raw')]+'.pdf')

    dfs.append(df)
    df.to_csv(rootDir + file, sep='\t', index=0)


  0%|          | 0/55 [00:00<?, ?it/s]


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [24]:
df = pd.concat(dfs)
df.index = pd.Index(range(len(df)))
df.to_csv('bkk-bud_raw.tsv', sep='\t')